<a href="https://colab.research.google.com/github/Insight-Sogang-Univ/insight-14th/blob/jiwon/advanced/jiwon/session05/assignment_languagemodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/Insight-Sogang-Univ/insight-14th/blob/main/advanced/template/session05/assignment_languagemodel.ipynb" target="_parent">
      <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
  </td>
</table>


# 🔍 이론 문제

### 1. SLM(통계적 언어모델)의 한계에 대해 설명하세요.

**[1번 문제 정답]**

***여기에 정답을 써주세요!*** \
희소성 문제: 문장이 길어질수록, 그 문장 안의 특정 표현을 담은 텍스트를 원본 학습 데이터에서 찾기가 어려워진다.  
-> 그 표현이 학습 데이터에 없다면, 확률을 계산할 수 없어지는 문제

### 2. BERT에 대한 설명이 아닌 것은?

a) 사전 학습 모델

b) 양방향 문맥 이해

c) 트랜스포머의 디코더를 쌓아 올린 구조

d) 이후 RoBERTa와 ALBERT로 발전

**[2번 문제 정답]**

***여기에 정답을 써주세요!*** \
c: BERT는 인코더를 사용한다.

### 3. BERT와 GPT의 학습 방식 차이를 간단히 설명하세요.

**[3번 문제 정답]**

***여기에 정답을 써주세요!*** \
BERT: 트랜스포머 인코더를 사용, MLM과 NSP를 통해 양방향 문맥을 위주로 학습  
GPT: 트랜스포머 디코더를 사용, 단방향성으로, 다음에 올 확률이 가장 높은 단어를 선택하면서 문장을 생성

### 4. LLM이 지닌 한계와 RAG의 필요성에 대해 간단히 서술하세요.

**[4번 문제 정답]**

***여기에 정답을 써주세요!*** \
환각(할루시) 문제: LLM이 특정 분야나 지엽적인 주제에 대해 모를 때는 말을 그럴 듯하게 지어내는 현상

RAG의 필요성: 수많은 데이터를 학습시켜도 환각은 발생함. 그리고 새로 학습시키는 것은 천문학적인 비용을 요구함. 따라서 관련된 정보를 찾고+LLM과 함께 답변 생성 하는 파이프라인을 만든 것

### 5. LLM을 경량화한 모델인 sLM을 만들기 위해 사용되는 대표적인 기술 3가지를 적으세요.(단답형)

**[5번 문제 정답]**

***여기에 정답을 써주세요!*** \
양자화, 가지치기, 지식 증류

# ✍ 언어 모델 과제

### 🧮 GPU 사용하기

In [ ]:
import torch
import tensorflow as tf

# PyTorch GPU 상태
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ [PyTorch] Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")


# TensorFlow GPU 상태
print("\n✅ [TensorFlow] GPU Devices:")
print(tf.config.list_physical_devices('GPU'))

✅ [PyTorch] Using device: cuda
GPU name: Tesla T4

✅ [TensorFlow] GPU Devices:
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 🔍 1. BERT

**BERT (Bidirectional Encoder Representations from Transformers)**
- 문장 분류(스팸 메일 탐지), 질의응답 시스템(챗봇, 검색 엔진), 번역(다국어 지원), 텍스트 요약(뉴스 요약) 등 다양한 NLP 작업에 사용
- 문맥을 양방향으로 이해하여 텍스트의 의미를 정밀하게 파악하며, 사전 학습된 모델을 기반으로 빠르게 응용 가능

### 1-1. 모델과 tokenizer 초기화
- tokenizer를 통해 문장을 토큰으로 나누고, 이를 정수 인덱스로 변환하여 모델이 이해할 수 있도록 함

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# BERT 모델과 tokenizer 초기화
tokenizer = BertTokenizer.from_pretrained('klue/bert-base') # base : 모델크기, (uncased : 소문자 학습)
model = BertForMaskedLM.from_pretrained('klue/bert-base') # base : 모델크기, (uncased : 소문자 학습)
# tokenizer : raw text를 개별 토큰으로 분리(Wordpiece). 토크나이저 사전에 따라 고유한 정수 인덱스(고정값)를 매핑함.

# tokenizer를 통해 생성된 정수 인덱스 : 텍스트를 모델이 읽을 수 있도록 숫자화
# Embedding vector : 단어의 의미적, 문맥적 특성을 모델링

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 1-2. 입력 텍스트 준비 및 토큰화
- [MASK] 토큰을 삽입한 문장을 설정하고, `tokenizer.tokenize()`로 텍스트 토큰화
- [MASK]를 활용하여 문맥 속에서 특정 단어를 추론하는 상황을 만들고, 이를 모델이 학습한 패턴과 비교하도록 함


In [ ]:
# 테스트할 문장
text = "[CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]"

# 문장을 토큰으로 변환
tokenized_text = tokenizer.tokenize(text) # text(문장)을 토큰으로 변환해 tokenized_text에 저장합시다!
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) # 토큰을 index로 변환하는 코드!

print(tokenized_text) # 토큰이 출력됩니다.
print(indexed_tokens) # 인덱스가 출력됩니다.


['[CLS]', '열', '##여', '##덟', ',', '우리', '##는', '서로', '##의', '이름', '##을', '처음', '불렀', '##다', '.', '그리고', '스물', '하나', ',', '우린', '[MASK]', '을', '했', '##다', '.', '[SEP]']
[2, 1432, 2173, 3542, 16, 3616, 2259, 4084, 2079, 3934, 2069, 3790, 6895, 2062, 18, 3673, 10514, 3657, 16, 8983, 4, 1498, 1902, 2062, 18, 3]


### 1-3. MASK된 위치 확인
- [MASK] 토큰의 위치를 탐지해 해당 위치에서 모델이 단어를 예측하도록 지정합니다.


In [ ]:
# 마스킹된 위치 찾기
masked_index = tokenized_text.index("[MASK]") # 토큰화된 결과물에서 index 메서드를 통해 [MASK]의 위치를 확인해봅시다!
print(masked_index)

20


### 1-4. 정수화된 토큰 인덱스를 tensor로 변환 후 모델 예측 실행
- 모델은 **텐서 데이터 구조**를 필요로 하므로, **입력값을 변환**하여 적합한 형식으로 만듦.
   - `torch.tensor(입력값 리스트 또는 입력값 ndarray)`를 통해 tensor 구조로 변환 가능!
- **역전파를 비활성화**하고, 모델이 **각 토큰에 대해 가능한 모든 단어 점수 출력**

In [ ]:
# 토큰화된 텍스트 -> 정수인덱스(ID) -> Pytorch 텐서
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) # 토큰을 index로 변환하는 코드!
tokens_tensor = torch.tensor([indexed_tokens]) # 입력값(토큰 인덱스 리스트)을 tensor 구조로 변환해봅시다!

# 모델에 토큰 텐서를 전달하고 예측 실행
with torch.no_grad(): # 가중치 업데이트 없으므로 자동미분연산은 끔
    outputs = model(tokens_tensor) # 예측. 모델은 각 토큰위치에 대한 예측 결과를 반환.
    predictions = outputs[0] # logits (각 토큰위치에서 가능한 모든 토큰들의 원시점수)
print(predictions)

tensor([[[ -6.0337,   4.5526,  -5.6312,  ...,  -7.3959,  -7.4220,  -5.8337],
         [ -6.1762,   4.7585,  -7.3973,  ...,  -7.6209, -12.2124,  -6.0998],
         [ -7.4194,   3.9148,  -6.1517,  ...,  -6.8162,  -9.5421,  -3.0473],
         ...,
         [ -7.3761,   8.6972,  -5.4904,  ...,  -8.7202,  -9.1403,  -5.8566],
         [ -5.6347,  10.3884,  -4.3374,  ...,  -8.6900,  -7.4171,  -3.5043],
         [ -5.6652,  10.3407,  -4.4099,  ...,  -8.7521,  -7.4286,  -3.6681]]])


### 1-5. MASK된 토큰 예측
- [MASK] 위치에서 가장 높은 점수를 받은 토큰의 인덱스를 추출하고, 이를 단어로 변환

In [ ]:
# 예측된 토큰 확인
predicted_index = torch.argmax(predictions[0, masked_index]).item() # masked_index에 들어갈 것으로 가장 확률이 높은 인덱스
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

# 예측된 토큰으로 마스크 채우기
tokenized_text[masked_index] = predicted_token
# 토큰화된 텍스트를 다시 문자열로 변환
filled_text = tokenizer.convert_tokens_to_string(tokenized_text[1:-1])

print("Original:", text)
print("Masked:", tokenized_text)
print("Predicted token:", predicted_token) ; print()

print("Filled sentence:", filled_text)

Original: [CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]
Masked: ['[CLS]', '열', '##여', '##덟', ',', '우리', '##는', '서로', '##의', '이름', '##을', '처음', '불렀', '##다', '.', '그리고', '스물', '하나', ',', '우린', '사랑', '을', '했', '##다', '.', '[SEP]']
Predicted token: 사랑

Filled sentence: 열여덟 , 우리는 서로의 이름을 처음 불렀다 . 그리고 스물 하나 , 우린 사랑 을 했다 .



### 1-6. MASK된 문장 복원
- 예측된 토큰으로 [MASK]를 대체하여 완성된 문장 생성

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# BERT 모델과 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
model = BertForMaskedLM.from_pretrained('klue/bert-base')

def fill_mask(input_text):
    # 텍스트를 토큰으로 변환
    tokenized_text = tokenizer.tokenize(input_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # 마스킹된 위치 찾기
    masked_index = tokenized_text.index("[MASK]") # 토큰화된 텍스트에서 마스킹된 위치를 찾아봅시다!

    # 토큰화된 텍스트 -> 정수인덱스(ID) -> Pytorch 텐서
    tokens_tensor = torch.tensor([indexed_tokens])

    # 모델에 토큰 텐서를 전달하고 예측 수행
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    # 예측된 토큰 확인
    predicted_index = torch.argmax(predictions[0, masked_index]).item() # 위 코드 참고! masked_index에 들어갈 것으로 가장 확률이 높은 인덱스
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

    # 마스크를 채운 문장 반환
    tokenized_text[masked_index] = predicted_token # 토큰화된 텍스트의 마스킹된 위치(인덱스 사용)를 예측된 토큰으로 채워봅시다!
    return tokenizer.convert_tokens_to_string(tokenized_text[1:-1])

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


예시 문장 만들어서 MLM을 사용해 보세요!

In [ ]:
# 예시 1: "[CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]"
# 예시 2: "[CLS] 윤해민은 INSGIHT 학회를 정말 사랑한다. 그는 INSIGHT 학회를 위해 [MASK]를 했다.[SEP]"
# 이 예시를 참고해서 [CLS](맨 앞), [MASK](채울 부분), [SEP](맨 뒤)를 추가해서 여러분만의 예시 문장 만들어주세요
input_text = "[CLS] 시중가보다 저렴하게 사서 너무 좋아요 집들이할때 썼는데 너무 이쁘더라구요 다른 [MASK]도 사고싶네요 세일 또 기다릴게요 [SEP]"
filled_text = fill_mask(input_text)
print("Filled sentence:", filled_text)

Filled sentence: 시중가보다 저렴하게 사서 너무 좋아요 집들이할때 썼는데 너무 이쁘더라구요 다른 제품 도 사고싶네요 세일 또 기다릴게요


# 🔍 2. GPT, RAG, Langchain

## 2-1 라이브러리 설치

langchain 관련 라이브러리와, pdf 문서를 읽을수 있는 pypdf 라이브러리를 설치할게요!

In [ ]:
# ⬇️ Colab에서 한 번만 실행
%pip install -qU "langchain>=0.3" langchain-community langchain-openai langgraph chromadb pypdf tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 38.6 MB/s eta 0:0

## 2-2. 키값 설정

## 2-3 pdf 문서 선택 (로드)


여러분들이 활용하고 싶은 아무 pdf 문서를 선택해보세요!

In [ ]:
from google.colab import files
import os, re, unicodedata

os.makedirs("/content/pdfs", exist_ok=True)
print("📥 업로드할 PDF 파일을 선택하세요 (여러 개 가능).")
uploaded = files.upload()

name_map = {}
for i, (orig_name, data) in enumerate(uploaded.items(), 1):
    nf = unicodedata.normalize("NFC", orig_name)
    base, ext = os.path.splitext(nf)
    # 한글/영문/숫자/밑줄/대시/점만 남기고 나머지는 _
    safe_base = re.sub(r"[^\w\u3131-\u318E\uAC00-\uD7A3.\-]+", "_", base).strip("._'‘’“”")
    # 파일명 너무 길면 잘라냄 (최대 ~140자 정도로)
    safe_base = safe_base[:120] if len(safe_base) > 120 else safe_base
    new_name = f"doc_{i}_{safe_base}{ext.lower()}"
    with open(os.path.join("/content/pdfs", new_name), "wb") as f:
        f.write(data)
    name_map[new_name] = orig_name

print("✅ 저장 완료 (Colab 파일명 → 원본명 매핑):")
for k, v in name_map.items():
    print(f"  {k}  <=  {v}")


📥 업로드할 PDF 파일을 선택하세요 (여러 개 가능).


Saving KBB_SCHOLAR_별점 평가 중심 대중음악 평론 방식 보편화에 대한 비판적 검토.pdf to KBB_SCHOLAR_별점 평가 중심 대중음악 평론 방식 보편화에 대한 비판적 검토.pdf
✅ 저장 완료 (Colab 파일명 → 원본명 매핑):
  doc_1_KBB_SCHOLAR_별점_평가_중심_대중음악_평론_방식_보편화에_대한_비판적_검토.pdf  <=  KBB_SCHOLAR_별점 평가 중심 대중음악 평론 방식 보편화에 대한 비판적 검토.pdf


## 2-4 문서 분할 후 벡터 DB에 저장
로딩한 문서를 분할 할 때,
- chunk_size는 `900`
- chunk_overlap은 `150`
- separators(구분자)는 `["\n\n", "\n", " ", ""]`
으로 설정하여 청킹해주세요!

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import glob

# 1) PDF → Documents
pdf_paths = sorted(glob.glob("/content/pdfs/*.pdf"))
docs = []
for p in pdf_paths:
    try:
        loader = PyPDFLoader(p)
        docs.extend(loader.load())
    except Exception as e:
        print(f"⚠️ 로딩 실패: {p} -> {e}")

print(f"총 문서 조각(페이지 기준): {len(docs)}")

# 2) 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 900, # 청크 사이즈: 문서를 몇 개의 토큰 단위로 나눌 것인지를 정합니다.
    chunk_overlap= 150, # 청크 오버랩: 분할된 끝 부분에서 맥락이 이어질 수 있도록 일부를 겹쳐서(overlap) 분할하는 것이 일반적입니다.
    separators= ["\n\n", "\n", " ", ""], # 구분자: 엔터, 공백 기준으로 나눔
)
chunks = text_splitter.split_documents(docs)
print(f"총 청크 수: {len(chunks)}")

# 3) 벡터스토어 (영속 디렉토리 쓰면 런타임 재시작해도 유지)
persist_dir = "/content/chroma_pdf_db"
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 비용 절약형
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=persist_dir
)
retriever = vectordb.as_retriever(search_kwargs={"k": 5})
print("✅ 인덱싱 완료")


총 문서 조각(페이지 기준): 41
총 청크 수: 69
✅ 인덱싱 완료


## 2-5 RA<U>Generate</U> : 생성 부분 선언

PROMPT 부분은 여러분들이 원하시는대로 입력해도 좋습니다!

🤥 영어로 입력하면 모델이 더 잘 이해한대요

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

PROMPT = ChatPromptTemplate.from_template(
"""다음 컨텍스트(업로드한 PDF에서 추출됨)만을 바탕으로 질문에 한국어로 답하라.
불확실하면 모른다고 말하라. 핵심 근거를 요약하고, 각 근거 옆에 출처(파일/페이지)를 표시하라.

[질문]
{question}

[컨텍스트]
{context}
"""
)

rag_chain = (PROMPT | llm | StrOutputParser())


## 2-6 RAG 파이프라인 langgraph로 구성

벡터db에서 검색하는 것이 <U>Retrieve</U>AG이고,

검색 결과를 생성 단계에 던져주는 것이 R<U>Augment</U>G입니다!

In [ ]:
from typing import TypedDict, List
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END

# format_docs의 역할은 문서를 포맷팅해서 모델에게 제공할 'context'를 정리하는 역할입니다!
def format_docs(docs):
    # 파일명/페이지를 메타데이터에 담아두는 PyPDFLoader 기본값 사용
    lines = []
    for i, d in enumerate(docs, 1):
        src = d.metadata.get("source", "")
        page = d.metadata.get("page", None)
        tag = f"{os.path.basename(src)}"
        if page is not None:
            tag += f" p.{page+1}"
        lines.append(f"[{i}] {tag}\n{d.page_content}")
    return "\n\n".join(lines)

# 그래프 선언
class QAState(TypedDict):
    question: str
    retrieved: List[Document]
    answer: str

def node_retrieve(state: QAState):
    q = state["question"]
    topk = retriever.invoke(q)
    return {"retrieved": topk}

def node_generate(state: QAState):
    q = state["question"]
    ctx = format_docs(state["retrieved"])
    ans = rag_chain.invoke({"question": q, "context": ctx})
    return {"answer": ans}

# --------- 그래프 구성 ---------
# 노드끼리 연결하기
workflow = StateGraph(QAState)
workflow.add_node("retrieve", node_retrieve) # 어떤 노드가 추가되어야 할까요?
workflow.add_node("generate", node_generate) # 어떤 노드가 추가되어야 할까요?

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate") # 어떤 노드와 어떤 노드가 연결되어야 할까요?
workflow.add_edge("generate", END)

app = workflow.compile()
print("✅ LangGraph 컴파일 완료")


✅ LangGraph 컴파일 완료


## 2-7 실제 구동해보기

question에 여러분들이 입력하고 싶은 질문을 넣어보세요!

RAG 과제는 이렇게 간단하게 끝내겠습니다😁

In [ ]:
question = "이 논문의 서론 및 선행연구 부분을 범주화하여 정리해줘."
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


▼ 답변
이 논문의 서론 및 선행연구 부분을 다음과 같이 범주화하여 정리할 수 있습니다.

1. **평론의 권위와 변화**
   - 평론의 권위와 영향력이 약화되고 있으며, 이는 대중음악뿐만 아니라 문학, 영화, 미술 등 다양한 분야에서 나타나고 있다. (출처: [1] p.3)

2. **평론의 위기와 원인**
   - 평론의 위기는 평단의 폐쇄적 성격, 논쟁적 성격의 상실, 공적 장의 부재 등 여러 원인으로 분석되고 있다. (출처: [1] p.3)

3. **별점 평가의 비판적 고찰**
   - 본 논문은 별점 평가 방식이 대중음악 평론에서 보편화된 현상을 비판적으로 살펴보며, 이를 통해 현대 사회의 사고방식과 가치체계와 연결지어 논의하고자 한다. (출처: [3] p.2)

4. **평론의 역사적 변화**
   - 평론의 역사적 변화와 현재의 변화가 지닌 의미를 추적하며, 평론이 어떻게 전문성을 인정받고 존재 가치를 증명해 왔는지를 살펴본다. (출처: [3] p.2)

5. **미디어 환경의 변화**
   - 2000년대 이후 미디어 환경의 변화가 평론의 형식과 내용에 미친 영향을 분석하고, 별점 평가의 부상과 그로 인한 대중음악 평론의 경향 변화를 논의한다. (출처: [5] p.4)

이와 같은 범주화는 논문의 서론 및 선행연구 부분에서 다루고 있는 주요 주제와 논점을 정리한 것입니다.


In [ ]:
question = "이 논문에 대해서 요약해줘"
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


▼ 답변
이 논문은 "별점 평가 중심 대중음악 평론 방식 보편화에 대한 비판적 검토"라는 제목을 가지고 있으며, 대중음악 평론에서 별점 평가 방식의 보편화에 대해 비판적으로 분석하고 있다. 주요 내용은 다음과 같다.

1. **평론의 변화**: 평론의 권위와 영향력이 감소하고 있으며, 이는 대중음악뿐만 아니라 문학, 영화, 미술 등 다양한 예술 분야에서 나타나는 현상이다. 이러한 변화는 미디어 환경의 변화와 예술에 대한 사회문화적 시선의 변화에 기인한다. (출처: [5] p.3)

2. **별점 평가의 문제점**: 별점 평가 방식은 대중음악 평론에서 일반화되고 있지만, 이는 평가의 질을 저하시킬 수 있으며, 수치로 환산된 평가가 질적 평가보다 더 신뢰받는 현대 사회의 사고 방식과 연결되어 있다. (출처: [5] p.3)

3. **대중음악 평론의 역사적 관점**: 대중음악 평론의 역사적 변화와 수사의 미학을 살펴보며, 별점 평가가 어떻게 대중음악 평론에 자리 잡게 되었는지를 분석한다. (출처: [1] p.1)

4. **일반 수용자와 전문 평론가의 평가 차이**: 일반 수용자와 전문 평론가 간의 평가 방식과 그 결과의 차이를 논의하며, 평가의 주관성과 객관성 간의 갈등을 다룬다. (출처: [4] p.9)

5. **결론**: 논문은 별점 평가 방식이 대중음악 평론의 질적 저하를 초래할 수 있음을 경고하며, 새로운 평론의 가능성을 모색할 필요성을 강조한다. (출처: [5] p.3)

이 논문은 대중음악 평론의 현재와 미래에 대한 깊이 있는 논의를 제공하며, 별점 평가 방식의 비판적 검토를 통해 평론의 새로운 방향성을 제시하고자 한다.


In [ ]:
question = "이 논문에서 나타나는 기성 대중음악 평론과, 최근 드러나는 개인 리스너 평론(예: 인스타그램 큐레이팅 계정, RYM)에 대한 너의 관점을 제시해줘."
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


▼ 답변
이 논문에서는 기성 대중음악 평론과 개인 리스너 평론의 차이점과 그로 인한 문제점을 다루고 있습니다. 기성 대중음악 평론은 전문가의 주관적 평가가 개입되며, 일반 수용자와의 평가 차이가 존재하는 경향이 있습니다. 반면, 개인 리스너 평론은 소셜 미디어 플랫폼을 통해 더 많은 사람들의 의견을 반영할 수 있는 장점이 있습니다.

1. **기성 대중음악 평론의 한계**: 전문가 평론가는 일반 수용자보다 다양한 관점에서 음악을 분석하지만, 주관이 개입될 가능성이 있으며, 경제적 성과를 충분히 설명하지 못하는 경우가 많습니다. 이로 인해 전문가 평론과 일반 수용자 간의 점수 차이가 발생합니다. (출처: [4] p.29)

2. **개인 리스너 평론의 장점**: 개인 리스너들은 소셜 미디어를 통해 자신의 의견을 쉽게 표현할 수 있으며, 이는 다양한 의견을 수렴하는 데 유리합니다. 특히 인스타그램 큐레이팅 계정이나 RYM과 같은 플랫폼은 개인의 주관적인 경험을 바탕으로 한 평론을 가능하게 합니다. (출처: [5] p.4)

3. **신뢰성 문제**: 기성 평론가의 신뢰성이 의심받는 경우가 있으며, 이는 개인 리스너들이 더 많은 신뢰를 받을 수 있는 환경을 조성합니다. 예를 들어, 게임 평론가에 대한 신뢰가 하락한 사례가 언급되며, 이는 음악 평론에도 유사한 경향이 나타날 수 있음을 시사합니다. (출처: [4] p.29)

결론적으로, 기성 대중음악 평론은 전문가의 주관적 평가가 개입되어 신뢰성에 문제가 있을 수 있지만, 개인 리스너 평론은 더 다양한 의견을 반영할 수 있는 장점이 있습니다. 그러나 개인 평론의 신뢰성 또한 검토가 필요합니다.


# 🔍 3. sLM

Copyright 2024 Google LLC.

## 3-1. sLM(Gemma) 로드
### sLM(small Language Model)의 대표 모델인 Gemma를 로드 해봅시다!

### (1) 사전 준비 사항
### 1. HuggingFace 로그인

https://huggingface.co/

이 링크로 들어가서 HuggingFace에 로그인합니다.(계정이 없다면 회원가입 해주세요!)

### 2. 토큰 발급받기

https://huggingface.co/settings/tokens

- 이 링크로 들어가서 'Create new token'을 클릭
- Token type: 'Read'로 해주세요.
- Token name: 아무거나 상관 없습니다.
- (⭐ 매우 중요!) 토큰 만드시고 **반드시 토큰 문자열 (hf_... 로 시작함)을 복사해주세요!**  
한 번만 볼 수 있습니다!

### 3. google colab에서 토큰 등록하기
- 왼쪽에 열쇠 모양 버튼(보안 비밀)을 눌러주세요.
- 이름: colab-gemma
- 값: 복사해둔 토큰 문자열(hf_...로 시작함)
- 노트북 액세스 : 스위치 키기(파란색)

### 4. gemma-2b-it 모델 사용 약관 동의

https://huggingface.co/google/gemma-2b-it

- (⭐ 매우 중요!) 반드시 2번 단계에서 토큰을 발급받았던 바로 그 Hugging Face 계정으로 로그인된 상태에서 위 링크로 들어가야 합니다.
- 링크로 들어가서 'Access Gemma on Hugging Face' 부분의 약관을 읽고 동의 버튼을 클릭해 주세요.
- "You have been granted access to this model"이라는 메시지가 뜨면 성공입니다.

### (2) 필요한 툴 설치

In [ ]:
!pip install -U bitsandbytes
!pip install --upgrade -q transformers huggingface_hub peft \
  accelerate bitsandbytes datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 17.6 MB/s eta 0:00:00


In [ ]:
# HuggingFace에 로그인합니다.

from huggingface_hub import login
from google.colab import userdata

# 따음표 안에 본인의 보안 비밀 키의 이름을 적습니다.(여기선 colab-gemma)
try:
    login(token=userdata.get("colab-gemma"))
except Exception as e:
    print(f"Error logging in to Hugging Face: {e}")
    print("Please make sure you have added your Hugging Face token to Colab secrets with the name 'colab-gemma'")

In [ ]:
# 모델을 로드합니다.
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 계산 부하를 줄이고 추론 속도를 높이기 위해 '양자화'합니다.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# sLM(gemma)를 Q&A, 요약 등에 활용하기 위해 '지시 튜닝(Instruction-Tuned) 버전의 모델을 사용합니다.
# "google/gemma-2b-it" 모델을 사용합니다.
model_id = "google/gemma-2b-it"

# 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델을 로드합니다.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config, # 양자화(quantization)
    device_map={"":"cuda:0"} # Changed from "0" to "cuda:0"
)

# GPU를 사용하도록 설정합니다.
device = "cuda"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

## 3-2. sLM 활용
- sLM은 LLM을 경량화한 모델이기 때문에 언어 모델의 기능인 질의응답(추론), 요약, 분류(감정분석) 등이 가능합니다.
- **속도는 비교적 빠르지만, 성능이 그닥 좋지 않다는 것을 확인하실 수 있습니다.**

### Gemma 활용 예시 (Common Use Cases)


### (1) 질의응답(Reasoning)

#### 원하는 질문을 한 번 넣어보세요!

In [ ]:
print("--- 예시 1: 질의응답 (Reasoning) ---")

# content : '~~' 따음표 안에 질문을 써주시면 됩니다.
# 하고 싶은 질문 아무거나 넣어보세요!

# "role": "user" 입니다.

chat = [
    # Gemma 모델의 채팅 형식에 맞춰 'role'과 'content'를 지정합니다.
    { "role": "user", "content": "시계열 분석의 AR, MA에 대해서 설명해줘" }
]

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성)
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)

--- 예시 1: 질의응답 (Reasoning) ---
**AR(Autoregressive) 모델**

AR 모델은 시계열 데이터를 과거 데이터를 기반으로 추론하는 모델입니다. AR 모델은 다음과 같은 과정을 통해 데이터를 예측합니다.

1. **입력 데이터**를 기반으로 과거 데이터를 추측합니다.
2. **예측된 값**을 현재 데이터에 추가합니다.
3. **오차**를 사용하여 과거 데이터와 현재 데이터의 관계를 모델링합니다.
4. **오차를 최소화**하는 과정을 통해 모델의 매개수를 추정합니다.

AR 모델은 다음과 같은 장점을 가지고 있습니다.

* **고품질 예측:** AR 모델은 과거 데이터를 기반으로 정확한 예측을 제공합니다.
* **자기회귀성:** AR 모델은 과거 데이터와 현재 데이터의 관계를 모델링하기 때문에, 자기적으로 예측될 수 있습니다.
* **다양한 데이터 형식:** AR 모델은 다양한 데이터 형식을 지원합니다.

**MA(Moving Average) 모델**

MA 모델은 시계열 데이터를 평균적으로 추추하는


### (2) 요약 (Summarization)

#### 원하는 글을 넣어서 요약해 보세요!

In [ ]:
print("\n--- 예시 2: 요약 (Summarization) ---")

# text_to_summarize = """~~~""" 따음표 안에 요약하고 싶은 글을 넣어주시면 됩니다.""
# 요약하고 싶은 글 아무거나 넣어보세요!

text_to_summarize = """
'크립토 윈터' 알트코인…ETF로 승부수
‘타이밍 악재’… 가상자산 전반 침체 영향
조정장에도 코인 ETF 줄대기 [쩐널리즘]
<앵커>
지난주 미국 셧다운 해제와 맞물려 나스닥에 카나리 캐피탈의 리플, XRP 상장지수펀드(ETF)가 상장을 했습니다. 하지만 기대와 달리 주춤하는 모습을 보였는데요. 출시를 앞둔 다른 가상자산 ETF에도 영향을 미치지 않을까 시장의 관심이 쏠리고 있습니다. 자세한 소식, 증권부 이민재 기자와 함께합니다. 이 기자, 기대가 컸던 XRP ETF가 힘을 잘 못 쓰는 이유, 뭡니까?

<기자>
가장 큰 원인은 시기입니다. 지금 가상자산 시장은 이른바 ‘크립토 윈터’, 즉 조정장에 대한 우려가 커지고 있습니다. 위험자산에 대한 선호가 전반적으로 줄어든 데다, 미국 연준의 매파적 발언과 트럼프 대통령의 레임덕 우려로 정책 불확실성이 커지고 있습니다. 여기에 유동성 감소와 가상자산 트레이딩 기업,(DAT) 등 이른바 ‘큰 손’들의 매도와 청산 물량이 쏟아지면서 악순환이 이어지고 있다는 분석입니다.

<앵커>
비트코인과 이더리움 같은 대표 가상자산 ETF들도 상황이 비슷합니까?

<기자>
비슷합니다. 비트코인 현물은 9만 달러선까지 밀리며 올해 상승분을 거의 반납했습니다. 업계에서는 단기 보유자들의 손실 규모가 2022년 FTX 사태 이후 최대 수준이라는 진단도 나옵니다. 이와 함께 관련 ETF에서 지난 12일(현지시각)부터 블랙록, 피델리티, 비트와이즈, 아크 등 주요 ETF 운용사들이 매도를 이어가며 자금 유출이 확대되고 있습니다. 하루 동안 약 8억 7천만 달러가 빠져나가는 등 ETF 출범 이후 역대 두 번째로 큰 규모의 유출이 발생하기도 했습니다. 알트코인 ETF 투자심리에도 찬물을 끼얹었습니다.

<앵커>
그런데도 여전히 상장을 준비 중인 가상자산 ETF가 많다고요. 이유가 뭡니까?

<기자>
약세라고 해서 자금이 안 들어오는 건 아닙니다. 실제로 리플 ETF는 첫날에만 3,500억 원 넘는 자금이 몰렸습니다. 이런 이유로 월가에서는 블랙록의 리플 ETF 참여 가능성이 거론되기도 했습니다. 리플보다 먼저 상장한 솔라나 ETF에는 10월 말부터 자금이 꾸준히 들어오고 있습니다.

<앵커>
결국 ETF 확대가 장기적으로는 시장 안정화에 도움이 된다고 볼 수 있을까요?

<기자>
그렇습니다. 가상자산 시장이 제도권의 신뢰를 얻는 게 핵심인데요.투자자들이 현물로 거래할 수 있는 가상자산을 굳이 ETF 형태로 거래하는 이유는, 자본시장 투자자들이 별도의 가상자산 거래소 가입이나 복잡한 절차, 그리고 코인 관련 불안감 없이 투자를 할 수 있도록 하기 위해섭니다. 즉, 가상자산을 ETF라는 그릇에 담아 자본시장에서 손쉽게 거래하도록 하는 것입니다. 이해하기 쉽게 다른 예를 들어보면 달러와 같은 법정화폐를 가상자산이라는 그릇에 담아 가상자산 시장에서 거래할 수 있도록 만든 것이 바로 스테이블코인입니다. 이들은 서로 시장의 유동성을 공급하는 역할을 합니다. 최근 다수의 ETF 출범과 스테이블코인 등장은 이러한 흐름의 기반이 되고 있습니다. 현재 상장을 앞둔 체인링크 현물 ETF를 비롯해 아발란체, 스텔라, 폴카닷, 도지코인, 시바이누 등 가상자산 ETF가 대기 중입니다. 여러 코인을 묶은 ‘지수형 ETF’ 출시도 예정돼 있습니다. 결국 이런 다양한 ETF들이 시장 문턱을 넘으려는 배경에는, 제도권 편입을 통한 신뢰 회복과 자금 유입의 선순환을 기대하는 시장 심리가 자리하고 있다는 분석입니다.

<앵커>
잘 들었습니다. 지금까지 증권부 이민재 기자였습니다.
"""

# content : '~~~'에 요청사항을 적으시면 됩니다. 여기서는 위 글을 세 문장으로 요약해 달라고 요청했습니다.
# "role": "user" 입니다.

chat = [
    { "role": "user", "content": f"Could you summarize the following text in three sentence?\n\nText:\n{text_to_summarize}" }
]

# 여기부터는 각 사례 모두 코드가 동일합니다.

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성).
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)


--- 예시 2: 요약 (Summarization) ---
1. A virtual asset exchange ETF가 상장을 했습니다.


2. 이유는 시기와 정책 불확실성이 커지고 있기 때문입니다.


3. ETF 확대가 장기적으로는 시장 안정화에 도움이 된다고 볼 수 있습니다.


### (3) 분류 (Classification)

원하는 문장을 넣고 감정 분석을 해보세요!

In [ ]:
print("\n--- 예시 3: 분류 (Classification) ---")

# content: '~~~' 따음표 안에 요청사항을 적으시면 됩니다.
# 여기서는 긍정, 부정, 중립으로 감정분석을 요청했습니다.

# \n\nText: 여기에 감정분석할 문장을 적어주시면 됩니다.
# 원하는 문장을 넣고 감정 분석을 해보세요!

# "role": "user" 입니다.

chat = [
    { "role": "user", "content": "Classify the text into neutral, negative, or positive. Generate only the class, nothing else.\n\nText: 기대했던 거 보단 별로인 거 같아요.. ㅠㅠ 처음에 이물질 하자로 교환하고 기다렸다가 받았구, 빠른 처리 주셨지만 가격에 비해 그냥 그런 거 같아요 ! " }
]

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성).
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)


--- 예시 3: 분류 (Classification) ---
Neutral: 기대했던 거 보단 별로인 거 같아요.

Negative: 처음에 이물질 하자로 교환하고 받았구, 빠른 처리 주셨지만 가격에 비해 그냥 그런 거 같아요 !



#### sLM 활용사례 결과 요약

- sLM은 질의응답이나 요약은 어느 정도 수행하지만, 퀄리티가 아쉽습니다.
- 감정 분석에서 '너무 재미가 없다'를 중립으로 판단하는 것으로 보아 분류 작업에서도 한계가 나타납니다.


- 이렇게 범용적인 활용에서는 한계가 나타나지만, 미세조정(Fine-Tuning)을 통해 특정 작업에 특화하면 해당 작업에는 좋은 성능을 내면서도 여전히 경량화된 모델의 장점을 누릴 수 있는 것이 sLM의 특징입니다!

# 🤩🤩 수고하셨습니다 🤩🤩
⭐️⭐️ 2-2의 API KEY 셀을 지우고 과제 제출해주세요!! ⭐️⭐️